# Lesson A5: Async Programming

**Duration**: 150-180 minutes  
**Stage**: Advanced (Mastery)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Understand async/await syntax and Future trait
2. Work with async runtimes (conceptually)
3. Handle async I/O operations
4. Combine async programming with error handling
5. Design async systems and understand their benefits

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. How do threads communicate using channels?
2. What's the purpose of Arc<T> and Mutex<T>?
3. What are Send and Sync traits?
4. How do you handle errors in concurrent code?

**Answers**: 1) mpsc channels for message passing, 2) Shared ownership and thread-safe mutation, 3) Thread safety markers, 4) Result types and proper error propagation

---

## 🧠 Key Concepts

### Async Programming Model

**Future Trait**: Represents a value that will be available in the future
**async/await**: Syntax for writing asynchronous code
**Runtime**: Executes async tasks (Tokio, async-std)
**Cooperative Multitasking**: Tasks yield control voluntarily
**Non-blocking I/O**: Operations don't block the thread

### Async vs Threads

- **Async**: Lightweight tasks, single-threaded by default, great for I/O
- **Threads**: OS-level parallelism, good for CPU-intensive work
- **Hybrid**: Combine both for optimal performance

---

## 🔬 Live Code Exploration

### Basic Async Functions and Futures

In [ ]:
// Basic async functions and futures
// Note: This is conceptual code for learning - evcxr doesn't support async runtimes

use std::future::Future;
use std::pin::Pin;
use std::task::{Context, Poll};
use std::time::{Duration, Instant};

// Simple async function
async fn simple_async_function() -> String {
    "Hello from async function!".to_string()
}

// Async function with delay simulation
async fn delayed_computation(delay_ms: u64, value: i32) -> i32 {
    // In real async code, this would be:
    // tokio::time::sleep(Duration::from_millis(delay_ms)).await;
    
    println!("Starting computation with {}ms delay for value {}", delay_ms, value);
    
    // Simulate async work
    value * 2
}

// Custom Future implementation
struct DelayedValue {
    value: i32,
    delay: Duration,
    start_time: Option<Instant>,
}

impl DelayedValue {
    fn new(value: i32, delay: Duration) -> Self {
        DelayedValue {
            value,
            delay,
            start_time: None,
        }
    }
}

impl Future for DelayedValue {
    type Output = i32;
    
    fn poll(mut self: Pin<&mut Self>, cx: &mut Context<'_>) -> Poll<Self::Output> {
        let start_time = self.start_time.get_or_insert_with(Instant::now);
        
        if start_time.elapsed() >= self.delay {
            println!("DelayedValue ready: {}", self.value);
            Poll::Ready(self.value)
        } else {
            // In a real runtime, we'd register a waker
            cx.waker().wake_by_ref();
            Poll::Pending
        }
    }
}

// Async error handling
#[derive(Debug)]
enum AsyncError {
    NetworkError(String),
    TimeoutError,
    ParseError(String),
}

impl std::fmt::Display for AsyncError {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        match self {
            AsyncError::NetworkError(msg) => write!(f, "Network error: {}", msg),
            AsyncError::TimeoutError => write!(f, "Operation timed out"),
            AsyncError::ParseError(msg) => write!(f, "Parse error: {}", msg),
        }
    }
}

impl std::error::Error for AsyncError {}

// Async function with error handling
async fn fetch_data(url: &str) -> Result<String, AsyncError> {
    println!("Fetching data from: {}", url);
    
    // Simulate different outcomes
    if url.contains("error") {
        Err(AsyncError::NetworkError("Connection failed".to_string()))
    } else if url.contains("timeout") {
        Err(AsyncError::TimeoutError)
    } else {
        Ok(format!("Data from {}", url))
    }
}

// Combining multiple async operations
async fn process_multiple_urls(urls: Vec<&str>) -> Vec<Result<String, AsyncError>> {
    let mut results = Vec::new();
    
    for url in urls {
        let result = fetch_data(url).await;
        results.push(result);
    }
    
    results
}

fn async_concepts_demo() {
    println!("=== Async Programming Concepts ===");
    
    // Note: In a real async environment, you would use:
    // #[tokio::main]
    // async fn main() {
    //     let result = simple_async_function().await;
    //     println!("Result: {}", result);
    // }
    
    println!("Async function concepts:");
    println!("- async fn returns impl Future<Output = T>");
    println!("- .await suspends execution until Future is ready");
    println!("- Futures are lazy - they don't run until awaited");
    println!("- Runtime manages task scheduling and I/O");
    
    // Demonstrate Future trait concepts
    println!("\nCustom Future implementation:");
    let delayed = DelayedValue::new(42, Duration::from_millis(100));
    println!("Created DelayedValue future (not yet executed)");
    
    // Error handling patterns
    println!("\nAsync error handling patterns:");
    println!("- Use Result<T, E> for fallible operations");
    println!("- ? operator works with async functions");
    println!("- Combine with try_join! for concurrent error handling");
}

async_concepts_demo();

### Async Patterns and Combinators

In [ ]:
// Async patterns and combinators (conceptual)

use std::collections::HashMap;

// Async data structures
#[derive(Debug, Clone)]
struct AsyncTask {
    id: u32,
    name: String,
    duration_ms: u64,
    dependencies: Vec<u32>,
}

impl AsyncTask {
    fn new(id: u32, name: String, duration_ms: u64) -> Self {
        AsyncTask {
            id,
            name,
            duration_ms,
            dependencies: Vec::new(),
        }
    }
    
    fn with_dependencies(mut self, deps: Vec<u32>) -> Self {
        self.dependencies = deps;
        self
    }
}

// Async task executor (conceptual)
struct AsyncTaskExecutor {
    tasks: HashMap<u32, AsyncTask>,
    completed: HashMap<u32, String>,
}

impl AsyncTaskExecutor {
    fn new() -> Self {
        AsyncTaskExecutor {
            tasks: HashMap::new(),
            completed: HashMap::new(),
        }
    }
    
    fn add_task(&mut self, task: AsyncTask) {
        self.tasks.insert(task.id, task);
    }
    
    // Simulate async task execution
    async fn execute_task(&mut self, task_id: u32) -> Result<String, String> {
        let task = self.tasks.get(&task_id)
            .ok_or_else(|| format!("Task {} not found", task_id))?;
        
        // Check dependencies
        for dep_id in &task.dependencies {
            if !self.completed.contains_key(dep_id) {
                return Err(format!("Dependency {} not completed for task {}", dep_id, task_id));
            }
        }
        
        println!("Executing task {}: {} ({}ms)", task.id, task.name, task.duration_ms);
        
        // In real async code:
        // tokio::time::sleep(Duration::from_millis(task.duration_ms)).await;
        
        let result = format!("Completed: {}", task.name);
        self.completed.insert(task_id, result.clone());
        
        Ok(result)
    }
    
    // Execute all tasks respecting dependencies
    async fn execute_all(&mut self) -> Vec<Result<String, String>> {
        let mut results = Vec::new();
        let task_ids: Vec<u32> = self.tasks.keys().cloned().collect();
        
        // Simple sequential execution (in real code, use proper dependency resolution)
        for task_id in task_ids {
            let result = self.execute_task(task_id).await;
            results.push(result);
        }
        
        results
    }
}

// Async stream processing (conceptual)
struct AsyncDataProcessor {
    name: String,
}

impl AsyncDataProcessor {
    fn new(name: String) -> Self {
        AsyncDataProcessor { name }
    }
    
    // Process data asynchronously
    async fn process_item(&self, item: i32) -> Result<i32, String> {
        println!("{}: Processing item {}", self.name, item);
        
        // Simulate processing time and potential errors
        if item < 0 {
            Err(format!("Invalid item: {}", item))
        } else {
            Ok(item * 2)
        }
    }
    
    // Process multiple items
    async fn process_batch(&self, items: Vec<i32>) -> Vec<Result<i32, String>> {
        let mut results = Vec::new();
        
        for item in items {
            let result = self.process_item(item).await;
            results.push(result);
        }
        
        results
    }
    
    // Concurrent processing (conceptual)
    async fn process_concurrent(&self, items: Vec<i32>) -> Vec<Result<i32, String>> {
        println!("{}: Starting concurrent processing of {} items", self.name, items.len());
        
        // In real async code with tokio:
        // let futures: Vec<_> = items.into_iter()
        //     .map(|item| self.process_item(item))
        //     .collect();
        // 
        // futures::future::join_all(futures).await
        
        // For demonstration, process sequentially
        self.process_batch(items).await
    }
}

// Async HTTP client simulation
struct AsyncHttpClient {
    base_url: String,
    timeout_ms: u64,
}

impl AsyncHttpClient {
    fn new(base_url: String, timeout_ms: u64) -> Self {
        AsyncHttpClient { base_url, timeout_ms }
    }
    
    async fn get(&self, path: &str) -> Result<String, AsyncError> {
        let url = format!("{}/{}", self.base_url, path);
        println!("GET {}", url);
        
        // Simulate network request
        if path.contains("slow") {
            // Simulate timeout
            Err(AsyncError::TimeoutError)
        } else if path.contains("error") {
            Err(AsyncError::NetworkError("404 Not Found".to_string()))
        } else {
            Ok(format!("Response from {}", url))
        }
    }
    
    async fn post(&self, path: &str, data: &str) -> Result<String, AsyncError> {
        let url = format!("{}/{}", self.base_url, path);
        println!("POST {} with data: {}", url, data);
        
        // Simulate POST request
        Ok(format!("Posted to {}: {}", url, data))
    }
    
    // Batch requests
    async fn batch_get(&self, paths: Vec<&str>) -> Vec<Result<String, AsyncError>> {
        let mut results = Vec::new();
        
        for path in paths {
            let result = self.get(path).await;
            results.push(result);
        }
        
        results
    }
}

fn async_patterns_demo() {
    println!("\n=== Async Patterns and Combinators ===");
    
    // Task execution patterns
    println!("Async task execution patterns:");
    println!("- Sequential: await each task in order");
    println!("- Concurrent: join_all for parallel execution");
    println!("- Racing: select! for first completion");
    println!("- Streaming: process data as it arrives");
    
    // Error handling patterns
    println!("\nAsync error handling:");
    println!("- try_join! fails fast on first error");
    println!("- join_all collects all results");
    println!("- timeout() adds time limits");
    println!("- retry() for resilient operations");
    
    // Common async patterns
    println!("\nCommon async patterns:");
    println!("- Producer/Consumer with async channels");
    println!("- Connection pooling for resources");
    println!("- Circuit breaker for fault tolerance");
    println!("- Rate limiting for API calls");
    
    // Performance considerations
    println!("\nPerformance considerations:");
    println!("- Async is great for I/O-bound tasks");
    println!("- Use threads for CPU-intensive work");
    println!("- Avoid blocking operations in async code");
    println!("- Consider task spawning overhead");
}

async_patterns_demo();

---

## 🎯 Guided Practice

### Exercise 1: Async Web Service Simulation

Create a simulation of an async web service with multiple endpoints.

In [ ]:
// TODO: Complete the async web service simulation

use std::collections::HashMap;
use std::time::{Duration, Instant};

#[derive(Debug, Clone)]
struct Request {
    id: u32,
    method: String,
    path: String,
    body: Option<String>,
    timestamp: Instant,
}

impl Request {
    fn new(id: u32, method: String, path: String) -> Self {
        Request {
            id,
            method,
            path,
            body: None,
            timestamp: Instant::now(),
        }
    }
    
    fn with_body(mut self, body: String) -> Self {
        self.body = Some(body);
        self
    }
}

#[derive(Debug, Clone)]
struct Response {
    status: u16,
    body: String,
    processing_time_ms: u64,
}

impl Response {
    fn ok(body: String, processing_time_ms: u64) -> Self {
        Response {
            status: 200,
            body,
            processing_time_ms,
        }
    }
    
    fn not_found(processing_time_ms: u64) -> Self {
        Response {
            status: 404,
            body: "Not Found".to_string(),
            processing_time_ms,
        }
    }
    
    fn error(message: String, processing_time_ms: u64) -> Self {
        Response {
            status: 500,
            body: message,
            processing_time_ms,
        }
    }
}

// Async web service
struct AsyncWebService {
    name: String,
    data_store: HashMap<String, String>,
    request_count: u32,
}

impl AsyncWebService {
    fn new(name: String) -> Self {
        let mut data_store = HashMap::new();
        data_store.insert("users/1".to_string(), "User 1 Data".to_string());
        data_store.insert("users/2".to_string(), "User 2 Data".to_string());
        data_store.insert("posts/1".to_string(), "Post 1 Content".to_string());
        
        AsyncWebService {
            name,
            data_store,
            request_count: 0,
        }
    }
    
    // Handle GET requests
    async fn handle_get(&mut self, path: &str) -> Response {
        let start_time = Instant::now();
        
        println!("{}: Handling GET {}", self.name, path);
        
        // Simulate processing time based on path
        let processing_delay = match path {
            p if p.contains("slow") => 200,
            p if p.contains("users") => 50,
            p if p.contains("posts") => 30,
            _ => 10,
        };
        
        // In real async code: tokio::time::sleep(Duration::from_millis(processing_delay)).await;
        
        let processing_time = start_time.elapsed().as_millis() as u64 + processing_delay;
        
        // Simulate different responses
        if path.contains("error") {
            Response::error("Internal Server Error".to_string(), processing_time)
        } else if let Some(data) = self.data_store.get(path) {
            Response::ok(data.clone(), processing_time)
        } else {
            Response::not_found(processing_time)
        }
    }
    
    // Handle POST requests
    async fn handle_post(&mut self, path: &str, body: &str) -> Response {
        let start_time = Instant::now();
        
        println!("{}: Handling POST {} with body: {}", self.name, path, body);
        
        // Simulate processing
        let processing_delay = 100; // POST operations are typically slower
        let processing_time = start_time.elapsed().as_millis() as u64 + processing_delay;
        
        if path.contains("error") {
            Response::error("Failed to create resource".to_string(), processing_time)
        } else {
            // Store the data
            self.data_store.insert(path.to_string(), body.to_string());
            Response::ok(format!("Created resource at {}", path), processing_time)
        }
    }
    
    // Main request handler
    async fn handle_request(&mut self, request: Request) -> Response {
        self.request_count += 1;
        
        println!("\n[Request {}] {} {} (ID: {})", 
                self.request_count, request.method, request.path, request.id);
        
        let response = match request.method.as_str() {
            "GET" => self.handle_get(&request.path).await,
            "POST" => {
                let body = request.body.as_deref().unwrap_or("");
                self.handle_post(&request.path, body).await
            }
            _ => Response::error("Method not allowed".to_string(), 1),
        };
        
        println!("[Response {}] Status: {}, Time: {}ms", 
                request.id, response.status, response.processing_time_ms);
        
        response
    }
    
    // Process multiple requests
    async fn process_requests(&mut self, requests: Vec<Request>) -> Vec<Response> {
        let mut responses = Vec::new();
        
        println!("\n🚀 Processing {} requests...", requests.len());
        
        for request in requests {
            let response = self.handle_request(request).await;
            responses.push(response);
        }
        
        responses
    }
    
    // Concurrent request processing (conceptual)
    async fn process_concurrent(&mut self, requests: Vec<Request>) -> Vec<Response> {
        println!("\n⚡ Processing {} requests concurrently...", requests.len());
        
        // In real async code with proper concurrency:
        // let futures: Vec<_> = requests.into_iter()
        //     .map(|req| self.handle_request(req))
        //     .collect();
        // futures::future::join_all(futures).await
        
        // For demonstration, process sequentially
        self.process_requests(requests).await
    }
    
    fn print_stats(&self, responses: &[Response]) {
        let total_requests = responses.len();
        let successful = responses.iter().filter(|r| r.status == 200).count();
        let errors = responses.iter().filter(|r| r.status >= 400).count();
        
        let total_time: u64 = responses.iter().map(|r| r.processing_time_ms).sum();
        let avg_time = if total_requests > 0 { total_time / total_requests as u64 } else { 0 };
        
        println!("\n📊 Service Statistics:");
        println!("  Total requests: {}", total_requests);
        println!("  Successful: {} ({:.1}%)", successful, 
                (successful as f64 / total_requests as f64) * 100.0);
        println!("  Errors: {} ({:.1}%)", errors, 
                (errors as f64 / total_requests as f64) * 100.0);
        println!("  Average response time: {}ms", avg_time);
        println!("  Total processing time: {}ms", total_time);
        println!("  Data store entries: {}", self.data_store.len());
    }
}

// Load balancer simulation
struct AsyncLoadBalancer {
    services: Vec<AsyncWebService>,
    current_service: usize,
}

impl AsyncLoadBalancer {
    fn new(service_names: Vec<String>) -> Self {
        let services = service_names.into_iter()
            .map(AsyncWebService::new)
            .collect();
        
        AsyncLoadBalancer {
            services,
            current_service: 0,
        }
    }
    
    // Round-robin load balancing
    async fn handle_request(&mut self, request: Request) -> Response {
        let service_index = self.current_service;
        self.current_service = (self.current_service + 1) % self.services.len();
        
        println!("🔄 Load balancer routing request {} to service {}", 
                request.id, service_index);
        
        self.services[service_index].handle_request(request).await
    }
    
    async fn process_requests(&mut self, requests: Vec<Request>) -> Vec<Response> {
        let mut responses = Vec::new();
        
        for request in requests {
            let response = self.handle_request(request).await;
            responses.push(response);
        }
        
        responses
    }
}

fn async_web_service_demo() {
    println!("\n=== Async Web Service Simulation ===");
    
    // Create requests
    let requests = vec![
        Request::new(1, "GET".to_string(), "users/1".to_string()),
        Request::new(2, "GET".to_string(), "users/2".to_string()),
        Request::new(3, "POST".to_string(), "users/3".to_string())
            .with_body("New User Data".to_string()),
        Request::new(4, "GET".to_string(), "posts/1".to_string()),
        Request::new(5, "GET".to_string(), "nonexistent".to_string()),
        Request::new(6, "GET".to_string(), "error-endpoint".to_string()),
        Request::new(7, "GET".to_string(), "slow-endpoint".to_string()),
        Request::new(8, "POST".to_string(), "posts/2".to_string())
            .with_body("New Post Content".to_string()),
    ];
    
    println!("Created {} test requests", requests.len());
    
    // Note: In a real async environment, you would run this with:
    // let mut service = AsyncWebService::new("TestService".to_string());
    // let responses = service.process_requests(requests).await;
    // service.print_stats(&responses);
    
    println!("\n💡 In a real async environment, this would:");
    println!("  - Process requests concurrently");
    println!("  - Use actual async I/O operations");
    println!("  - Handle thousands of concurrent connections");
    println!("  - Provide much better performance than blocking I/O");
    
    println!("\n🔧 Key async patterns demonstrated:");
    println!("  - Async functions with .await");
    println!("  - Error handling with Result types");
    println!("  - State management in async contexts");
    println!("  - Request/response patterns");
    println!("  - Load balancing and service distribution");
}

async_web_service_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. What is the Future trait and how does it work?
2. How does async/await syntax simplify asynchronous code?
3. What's the difference between async and threaded concurrency?
4. How do you handle errors in async functions?
5. What are the benefits of async programming for I/O?
6. How do async runtimes manage task execution?
7. When should you use async vs threads?
8. What are common async patterns and combinators?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **How does async programming change the way you think about I/O?**
2. **What are the trade-offs between async and traditional blocking I/O?**
3. **How do you design systems that effectively use async patterns?**
4. **What challenges does async programming introduce?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Macros & Metaprogramming

In our next lesson, you'll learn about:
- Declarative macros with macro_rules!
- Procedural macros and code generation
- Macro hygiene and best practices
- Building domain-specific languages

### How This Connects
Async programming is essential for:
- Building high-performance web services
- Handling concurrent I/O operations
- Creating scalable network applications
- Modern systems programming

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Understand the Future trait and async execution model?
- [ ] Write async functions with proper error handling?
- [ ] Choose between async and threaded approaches?
- [ ] Design async systems with appropriate patterns?
- [ ] Handle concurrent async operations effectively?
- [ ] Understand async runtime concepts?
- [ ] Apply async programming to real-world problems?

If you checked all boxes, excellent! You've mastered async programming fundamentals.

---

**🎉 Exceptional Mastery!** You now understand how to build high-performance async applications in Rust!